In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# Extract text from inside

def preprocess(folder):
    
    current_dir = directory + "/"+folder
    base_cases = []
    entailed_fragments = []
    is_entailing = []
    paragraphs = []
    list_of_files = sorted(os.listdir(current_dir))
    
    for i in range(len(list_of_files)-2):
        if i%100 == 0:
            print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        g = open(current_dir+'/'+list_of_files[i]+"/entailed_fragment.txt","r")
        h = open(current_dir+'/'+list_of_files[i]+"/entailing_paragraphs.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/paragraphs"))
        tmp = list(map(int,h.read().split("\n")[:-1]))
        a = f.read()
        b = g.read()
        for j in range(len(paragraph_filenames)):
            base_cases.append(a)
            entailed_fragments.append(b)
        f.close()
        g.close()
        h.close()
        
        arr = []
        for j in range(len(paragraph_filenames)):
            if j+1 in tmp:
                is_entailing.append(1)
            else:
                is_entailing.append(0)
        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/paragraphs/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,is_entailing
    
train_base_cases,train_entailed_fragments, train_paragraphs,train_is_entailing = preprocess('task2_train')


Processing  0
Processing  100


In [14]:
df = pd.DataFrame(data=np.column_stack([train_entailed_fragments,train_paragraphs,
                                      train_is_entailing]),columns=['question1','question2','is_duplicate'])

In [3]:
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('train.csv')
df = df.dropna(how="any").reset_index(drop=True)

In [ ]:
a = 0 
for i in range(a,a+10):
    print(df.question1[i])
    print(df.question2[i])
    print()

In [ ]:
question1 = 'What would a Trump presidency mean for current international master’s students on an F1 visa?'
question2 = 'How will a Trump presidency affect the students presently in US or planning to study in US?'

question1 = question1.lower().split()
question2 = question2.lower().split()

question1 = [w for w in question1 if w not in stop_words]
question2 = [w for w in question2 if w not in stop_words]

In [ ]:
# Word2Vec Model
data = df.question1.values + df.question2.values
model = gensim.models.Word2Vec(data,size=150,window=5,min_count=1,workers=10)
model.train(data, total_examples=len(data), epochs=25)

In [5]:
#model.save("train.bin.gz")
import gensim

from gensim.models import Word2Vec
    
model = gensim.models.KeyedVectors.load('quora.bin.gz')

2019-03-04 18:53:05,294 : INFO : loading Word2VecKeyedVectors object from quora.bin.gz
2019-03-04 18:53:05,448 : INFO : loading wv recursively from quora.bin.gz.wv.* with mmap=None
2019-03-04 18:53:05,450 : INFO : setting ignored attribute vectors_norm to None
2019-03-04 18:53:05,452 : INFO : loading vocabulary recursively from quora.bin.gz.vocabulary.* with mmap=None
2019-03-04 18:53:05,454 : INFO : loading trainables recursively from quora.bin.gz.trainables.* with mmap=None
2019-03-04 18:53:05,456 : INFO : setting ignored attribute cum_table to None
2019-03-04 18:53:05,457 : INFO : loaded quora.bin.gz


In [6]:
def wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return model.wmdistance(q1, q2)

In [7]:
def norm_wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return norm_model.wmdistance(q1, q2)

In [8]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [9]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)

In [15]:
df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
print("len_q1")
df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
print("len_q2")
df['diff_len'] = df.len_q1 - df.len_q2
df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
print("len_char_q1")
df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
print("len_char_q2")
df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
print("len_ wordq1")
df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
df['fuzz_ratio'] = df.apply(lambda x: fuzz.ratio(str(x['question1']), str(x['question2'])), axis=1)
df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
df['fuzz_token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
df['fuzz_token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

len_q1
len_q2
len_char_q1
len_char_q2
len_ wordq1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

In [25]:
#df.drop(['question1', 'question2'], axis=1, inplace=True)

#X = df.loc[:, df.columns != 'is_duplicate']
#y = df.loc[:, df.columns == 'is_duplicate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=0)


In [ ]:
X_train.head()

In [13]:
import xgboost as xgb

model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, 
                          gamma=0, reg_alpha=4, objective='binary:logistic', 
                          eta=0.3, verbosity=3, subsample=0.8).fit(X, y.values.ravel()) 
prediction = model.predict(X_test)
cm = confusion_matrix( y_test, prediction ) 
print(cm)  
print('Accuracy', accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

[[72378  4269]
 [ 4438 40202]]
Accuracy 0.9282115972857767
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     76647
           1       0.90      0.90      0.90     44640

   micro avg       0.93      0.93      0.93    121287
   macro avg       0.92      0.92      0.92    121287
weighted avg       0.93      0.93      0.93    121287



In [23]:
for i in range(500):
    print(prediction[i],i+1)

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184
0 18

In [40]:
prediction = model.predict(X_train)
y_train = Y_trr
cm = confusion_matrix( y_train, prediction ) 
print(cm)  
print('Accuracy', accuracy_score(y_train, prediction))
print(classification_report(y_train, prediction))

[[5765   11]
 [   0    0]]
Accuracy 0.998095567867036
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5776
           1       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00      5776
   macro avg       0.50      0.50      0.50      5776
weighted avg       1.00      1.00      1.00      5776



In [22]:
prediction[26]

0

In [38]:
Y_trr = []
for i in range(len(y_train)):
    Y_trr.append(int(y_train.values[i][1]))

In [37]:
y_train.values[0]

array(['0', 0], dtype=object)